# Symbolic Computation Speedups

This is a simple example of how symbolic computation can yield large reductions in the amount of computation.

In [ ]:
# Setup
import symforce
symforce.set_backend('sympy')

from symforce import geo
from symforce import sympy as sm
from symforce.notebook_util import display

# Autodiff vs Symbolic diff

### Multiplying three matrices (for example to chain rule Jacobians)

In [ ]:
A = geo.Matrix.zeros(5, 5).symbolic('A')
B = geo.Matrix.zeros(5, 3).symbolic('B')
C = geo.Matrix.zeros(3, 3).symbolic('C')
display(A, B, C)

In [ ]:
display(A * B * C)

In [ ]:
# Normal matrix multiplication (can happen with SIMD)
sm.count_ops(A * B * C)

### The same with typical sparsity patterns

In [ ]:
A = geo.Matrix.diag(sm.symbols('A:5'))
A[0, 4] = sm.Symbol('A5')
B = geo.Matrix.zeros(5, 3)
B[:3, :3] = geo.Matrix.diag(sm.symbols('B:3'))
B[3, 0] = sm.Symbol('B3')
B[3, 1] = sm.Symbol('B3')
B[3, 2] = sm.Symbol('B3')
B[4, 1] = sm.Symbol('B4')
C = geo.M(geo.Rot3.hat(sm.symbols('C:3')))
display(A, B, C)

In [ ]:
display(A * B * C)

In [ ]:
# Direct computation of result
sm.count_ops(A * B * C)

In [ ]:
# Looking at the sub-expressions
intermediates, output = sm.cse(A * B * C)

num_operations = 0
for lhs, rhs in intermediates:
    display(sm.Eq(lhs, rhs))
    num_operations += sm.count_ops(rhs)
    
output_mat = geo.Matrix53(output)
num_operations += sm.count_ops(output_mat)
display(output_mat)

In [ ]:
# With common sub-expression elimination
display(num_operations)

In [ ]:
# Code generation
for lhs, rhs in intermediates:
    print('float {} = {};'.format(lhs, rhs))

print('')
for i, out in enumerate(output):
    print('out[{}] = {};'.format(i, out))

Further points:

* simple example, entries are leaves
* only three matrices deep
* applies to other domains, not just derivatives + optimization
* not considering dynamic allocation (matrices on the heap), pointer chasing
* not considering you had to write the jacobians in the first place, test, debug